# Hyperparameter Tuning with NSGAIITuner

Tiny example of meta-optimizing NSGA-II hyperparameters on ZDT1. Budgets are intentionally small so the notebook runs quickly.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from vamos.problem.registry import make_problem_selection
from vamos.tuning import NSGAIITuner, build_nsgaii_config_space

plt.style.use("seaborn-v0_8")

In [ ]:
# Problem setup
selection = make_problem_selection("zdt1", n_var=10)
problem = selection.instantiate()
config_space = build_nsgaii_config_space(problem)

In [ ]:
# Configure a very small meta-optimization budget
tuner = NSGAIITuner(
    config_space=config_space,
    problems=[problem],
    ref_fronts=[None],
    indicators=["hv"],
    max_evals_per_problem=300,
    n_runs_per_problem=1,
    engine="numpy",
    meta_population_size=12,
    meta_max_evals=60,
    seed=0,
)
X_meta, F_meta, configs, diagnostics = tuner.optimize()
print("Meta front size:", F_meta.shape)

In [ ]:
# Build a small DataFrame of candidate configs
rows = []
for vec, obj, cfg in zip(X_meta, F_meta, configs):
    rows.append({"hv_objective": obj[0], "config": cfg})
df = pd.DataFrame(rows)
df.head()

In [ ]:
# Plot meta-objective values (remember: lower is better here because HV is negated)
plt.figure(figsize=(5, 3))
plt.scatter(range(len(df)), df["hv_objective"], c="tab:purple")
plt.xlabel("candidate")
plt.ylabel("meta objective (neg. HV)")
plt.title("Tuned NSGA-II configs (tiny budget)")
plt.tight_layout()
plt.show()